In [1]:
from pymavlink import mavutil

import time
import sys


# BASE SUR LE SITE
# https://www.ardusub.com/developers/pymavlink.html#send-rc-joystick

Voila ce que vous retrouverez sur ce lien :
- Autopilot (E.g: Pixhawk) connected to the computer via serial
- Run pyMavlink on the surface computer
- Run pyMavlink on the companion computer
- Send Message to QGroundControl
- Arm/Disarm the vehicle
- Change flight mode
- Send RC (Joystick)
- Send Manual Control
- Read all parameters
- Read and write parameters
- Receive data and filter by message type
- Request message interval
- Control Camera Gimbal
- Set Servo PWM
- Advanced Servo/Gripper Example
- Set Target Depth/Attitude
- Send GPS position
- Send rangefinder/computer vision distance measurement to the autopilot


### Connect to the SIM

In [10]:
master = mavutil.mavlink_connection('tcp:127.0.0.1:5762')  # Adjust IP and port if needed
print("Waiting for heartbeat...")
master.wait_heartbeat()
print("Heartbeat received! System ready.")

Waiting for heartbeat...


KeyboardInterrupt: 

### Wait for a ping

In [11]:
def wait_conn():
    """
    Sends a ping to stabilish the UDP communication and awaits for a response
    """
    msg = None
    while not msg:
        master.mav.ping_send(
            int(time.time() * 1e6), # Unix time in microseconds
            0, # Ping number
            0, # Request ping of all systems
            0 # Request ping of all components
        )
        msg = master.recv_match()
        time.sleep(0.5)

wait_conn()

print("Ping recu")

Ping recu


### Print every msg

In [4]:
for i in range(10):
    try:
        print(master.recv_match().to_dict())
    except:
        pass
    time.sleep(0.1)

{'mavpackettype': 'SYS_STATUS', 'onboard_control_sensors_present': 1399979055, 'onboard_control_sensors_enabled': 1382128687, 'onboard_control_sensors_health': 1467063343, 'load': 0, 'voltage_battery': 12600, 'current_battery': 0, 'battery_remaining': 100, 'drop_rate_comm': 0, 'errors_comm': 0, 'errors_count1': 0, 'errors_count2': 0, 'errors_count3': 0, 'errors_count4': 0}
{'mavpackettype': 'GPS_GLOBAL_ORIGIN', 'latitude': -353632621, 'longitude': 1491652374, 'altitude': 584090, 'time_usec': 49189483}
{'mavpackettype': 'HOME_POSITION', 'latitude': -353632621, 'longitude': 1491652374, 'altitude': 584090, 'x': 0.0, 'y': 0.0, 'z': -0.0, 'q': [nan, nan, nan, nan], 'approach_x': 0.0, 'approach_y': 0.0, 'approach_z': 0.0, 'time_usec': 49189483}
{'mavpackettype': 'PING', 'time_usec': 1737053172488433, 'seq': 0, 'target_system': 0, 'target_component': 0}
{'mavpackettype': 'HEARTBEAT', 'type': 6, 'autopilot': 8, 'base_mode': 192, 'custom_mode': 0, 'system_status': 4, 'mavlink_version': 3}
{'mav

### Send a message to QGroundControl

In [5]:
# Send a message for QGC to read out loud
#  Severity from https://mavlink.io/en/messages/common.html#MAV_SEVERITY
master.mav.statustext_send(mavutil.mavlink.MAV_SEVERITY_NOTICE,
                           "QGC will read this".encode())

### Arm and disarm

In [6]:
master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,
    1, 0, 0, 0, 0, 0, 0)

# wait until arming confirmed (can manually check with master.motors_armed())
print("Waiting for the vehicle to arm")
master.motors_armed_wait()
print('Armed!')

Waiting for the vehicle to arm


Armed!


In [7]:
# OR
master.arducopter_arm()

In [8]:
master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,
    0, 0, 0, 0, 0, 0, 0)

# wait until disarming confirmed
print("Waiting for the vehicle to disarm")
master.motors_disarmed_wait()
print('Disarmed!')

Waiting for the vehicle to disarm
Disarmed!


In [9]:
# OR
master.arducopter_disarm()

### Change Flight Mode

In [10]:
# Choose a mode
mode = 'STABILIZE'
# mode = 'GUIDED'

# Check if mode is available
if mode not in master.mode_mapping():
    print('Unknown mode : {}'.format(mode))
    print('Try:', list(master.mode_mapping().keys()))
    sys.exit(1)

# Get mode ID
mode_id = master.mode_mapping()[mode]

In [11]:
# Set new mode
# master.mav.command_long_send(
#    master.target_system, master.target_component,
#    mavutil.mavlink.MAV_CMD_DO_SET_MODE, 0,
#    0, mode_id, 0, 0, 0, 0, 0)
# OR
master.set_mode(mode_id)
# OR
# master.mav.set_mode_send(
#     master.target_system,
#     mavutil.mavlink.MAV_MODE_FLAG_CUSTOM_MODE_ENABLED,
#     mode_id)

while True:
    # Wait for ACK command
    # Would be good to add mechanism to avoid endlessly blocking
    # if the autopilot sends a NACK or never receives the message
    ack_msg = master.recv_match(type='COMMAND_ACK', blocking=True)
    ack_msg = ack_msg.to_dict()

    # Continue waiting if the acknowledged command is not `set_mode`
    if ack_msg['command'] != mavutil.mavlink.MAV_CMD_DO_SET_MODE:
        continue

    # Print the ACK result !
    print(mavutil.mavlink.enums['MAV_RESULT'][ack_msg['result']].description)
    break

Command is valid (is supported and has valid parameters), and was executed.


### Use RC

In [12]:
# Create a function to send RC values
# More information about Joystick channels
# here: https://www.ardusub.com/operators-manual/rc-input-and-output.html#rc-inputs

def set_rc_channel_pwm(channel_id, pwm=1500):
    """ Set RC channel pwm value
    Args:
        channel_id (TYPE): Channel ID
        pwm (int, optional): Channel pwm value 1100-1900
    """
    if channel_id < 1 or channel_id > 18:
        print("Channel does not exist.")
        return

    # Mavlink 2 supports up to 18 channels:
    # https://mavlink.io/en/messages/common.html#RC_CHANNELS_OVERRIDE
    rc_channel_values = [65535 for _ in range(18)]
    rc_channel_values[channel_id - 1] = pwm
    master.mav.rc_channels_override_send(
        master.target_system,                # target_system
        master.target_component,             # target_component
        *rc_channel_values)                  # RC channel list, in microseconds.

In [13]:
# Manual mode
mode = 'STABILIZE'
if mode not in master.mode_mapping():
    print('Unknown mode : {}'.format(mode))
    print('Try:', list(master.mode_mapping().keys()))
    sys.exit(1)
mode_id = master.mode_mapping()[mode]

In [81]:
# 1500 : joystick au centre (1100 et 1900)
while True:
    set_rc_channel_pwm(1, 1600)  # ROLL
    set_rc_channel_pwm(2, 1400)  # PITCH
    set_rc_channel_pwm(3, 1475)  # THROTTLE Monte doucement : 1475
    set_rc_channel_pwm(4, 1400)  # YAW
    time.sleep(0.1)

KeyboardInterrupt: 

### Controle manuel (imitaion d'une télécommande)

In [74]:
# Manual mode
mode = 'STABILIZE'
if mode not in master.mode_mapping():
    print('Unknown mode : {}'.format(mode))
    print('Try:', list(master.mode_mapping().keys()))
    sys.exit(1)
mode_id = master.mode_mapping()[mode]

In [85]:
# Send a positive x value, negative y, negative z,
# positive rotation and no button.
# https://mavlink.io/en/messages/common.html#MANUAL_CONTROL
# Warning: Because of some legacy workaround, z will work between [0-1000]
# where 0 is full reverse, 500 is no output and 1000 is full throttle.
# x,y and r will be between [-1000 and 1000].

# To active button 0 (first button), 3 (fourth button) and 7 (eighth button)
# It's possible to check and configure this buttons in the Joystick menu of QGC
buttons = 1 + 1 << 3 + 1 << 7

master.set_mode(mode)
while True:
    master.mav.manual_control_send(
        master.target_system,
        100, # PTICH
        100, # ROLL
        475, # THROTLLE (475 monte doucement)
        100, # YAW 
        buttons)
    time.sleep(0.1)

KeyboardInterrupt: 

### Read all parameters

In [80]:
master.mav.param_request_list_send(
    master.target_system, master.target_component
)
while True:
    time.sleep(0.01)
    try:
        message = master.recv_match(type='PARAM_VALUE', blocking=True).to_dict()
        print('name: %s\tvalue: %d' % (message['param_id'],
                                       message['param_value']))
    except Exception as error:
        print(error)
        sys.exit(0)

name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: STAT_RUNTIME	value: 1487
name: LAND_SPEED	value: 5
name: STAT_RUNTIME	value: 1518
name: STAT_RUNTIME	value: 1549
name: STAT_RUNTIME	value: 1580
name: STAT_RUNTIME	value: 1611
name: STAT_RUNTIME	value: 1642
name: STAT_RUNTIME	value: 1673
name: STAT_RUNTIME	value: 1704
name: STAT_RUNTIME	value: 1735
name: LAND_SPEED	value: 5
name: STAT_RUNTIME	value: 1766
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: LAND_SPEED	value: 5
name: FORMAT_VERSION	value: 120
name: SYSID_THISMAV	value: 1
name: SYSID_MYGCS	value: 255
name: PILOT_THR_FILT	value: 0
name: PILOT_TKOFF_ALT	value: 0
name: PILOT_THR_BHV	value: 0
name: TELEM_DELAY	value: 0
name: GCS_PID_MASK	value: 0
name: RTL_ALT	value: 1500
name: RTL_CONE_SLOPE	value: 3
name: RTL

KeyboardInterrupt: 

### Read and write parameters

Ici on va changer un paramètre de l'EEPROM qui définit le comportement de notre autopilote.

In [60]:
# Request parameter
master.mav.param_request_read_send(
    master.target_system, master.target_component,
    b'LAND_SPEED',
    -1
)

# Print old parameter value
message = master.recv_match(type='PARAM_VALUE', blocking=True).to_dict()
print('name: %s\nvalue: %d' %
      (message['param_id'], message['param_value']))

time.sleep(1)

name: LAND_SPEED
value: 50


In [77]:
# Set parameter value
# Set a parameter value TEMPORARILY to RAM. It will be reset to default on system reboot.
# Send the ACTION MAV_ACTION_STORAGE_WRITE to PERMANENTLY write the RAM contents to EEPROM.
# The parameter variable type is described by MAV_PARAM_TYPE in http://mavlink.org/messages/common.
master.mav.param_set_send(
    master.target_system, master.target_component,
    b'LAND_SPEED',
    5,
    mavutil.mavlink.MAV_PARAM_TYPE_REAL32
)

# Read ACK
# IMPORTANT: The receiving component should acknowledge the new parameter value by sending a
# param_value message to all communication partners.
# This will also ensure that multiple GCS all have an up-to-date list of all parameters.
# If the sending GCS did not receive a PARAM_VALUE message within its timeout time,
# it should re-send the PARAM_SET message.
message = master.recv_match(type='PARAM_VALUE', blocking=True).to_dict()
print('name: %s\nvalue: %d' %
      (message['param_id'], message['param_value']))

time.sleep(1)

name: LAND_SPEED
value: 5


In [79]:
# Request parameter value to confirm
master.mav.param_request_read_send(
    master.target_system, master.target_component,
    b'LAND_SPEED',
    -1
)

# Print new value in RAM
message = master.recv_match(type='PARAM_VALUE', blocking=True).to_dict()
print('name: %s\nvalue: %d' %
      (message['param_id'], message['param_value']))

name: LAND_SPEED
value: 5


### Recevoir des données et les triées par type de messaeges

In [81]:
# You can also filter within recv_match command - see "Read all parameters" example
while True:
    msg = master.recv_match()
    if not msg:
        continue
    if msg.get_type() == 'HEARTBEAT':
        print("\n\n*****Got message: %s*****" % msg.get_type())
        print("Message: %s" % msg)
        print("\nAs dictionary: %s" % msg.to_dict())
        # Armed = MAV_STATE_STANDBY (4), Disarmed = MAV_STATE_ACTIVE (3)
        print("\nSystem status: %s" % msg.system_status)
    time.sleep(0.1)



*****Got message: HEARTBEAT*****
Message: HEARTBEAT {type : 6, autopilot : 8, base_mode : 0, custom_mode : 0, system_status : 0, mavlink_version : 3}

As dictionary: {'mavpackettype': 'HEARTBEAT', 'type': 6, 'autopilot': 8, 'base_mode': 0, 'custom_mode': 0, 'system_status': 0, 'mavlink_version': 3}

System status: 0


*****Got message: HEARTBEAT*****
Message: HEARTBEAT {type : 2, autopilot : 3, base_mode : 81, custom_mode : 0, system_status : 3, mavlink_version : 3}

As dictionary: {'mavpackettype': 'HEARTBEAT', 'type': 2, 'autopilot': 3, 'base_mode': 81, 'custom_mode': 0, 'system_status': 3, 'mavlink_version': 3}

System status: 3


*****Got message: HEARTBEAT*****
Message: HEARTBEAT {type : 6, autopilot : 8, base_mode : 192, custom_mode : 0, system_status : 4, mavlink_version : 3}

As dictionary: {'mavpackettype': 'HEARTBEAT', 'type': 6, 'autopilot': 8, 'base_mode': 192, 'custom_mode': 0, 'system_status': 4, 'mavlink_version': 3}

System status: 4


*****Got message: HEARTBEAT*****

KeyboardInterrupt: 